In [1]:
%%bash
which python
python --version
#python -m ipykernel install --name py3.10-env --user
pip install -q tqdm openai elasticsearch pandas scikit-learn

/usr/local/python/current/bin/python
Python 3.10.8


**Restart jupuyter lab**

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:


In [9]:
# !docker kill $(docker ps -q -f name=elasticsearch)
!docker run --rm -it -d --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.4.3

3811f62315da074b7e464fcdd9a5abf0c316b988d3b92502bda39c53b6f83455


What's the `version.build_hash` value?

In [12]:
!curl -s localhost:9200 | jq .version.build_hash

"42f05b9372a9a4a470db3b52817899b99a76ee73"


## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [13]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [14]:
import json

_samples=json.dumps(documents[:int(len(documents) * 0.05):10],sort_keys=True, indent=4)
print(f"Docs count: {len(documents)}\nDocs samples: {_samples}")

Docs count: 948
Docs samples: [
    {
        "course": "data-engineering-zoomcamp",
        "question": "Course - When will the course start?",
        "section": "General course-related questions",
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel."
    },
    {
        "course": "data-engineering-zoomcamp",
        "question": "Course - \u200b\u200bHow many hours per week am I expected to spend on this  course?",
        "section": "General course-related questions",
        "text": "It depends on your background and previous experience with modules.

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [15]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 


In [16]:
es_client.info().body

{'name': '3811f62315da',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'nMroIMhcTLWBnXDN_5_HpQ',
 'version': {'number': '8.4.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
  'build_date': '2022-10-04T07:17:24.662462378Z',
  'build_snapshot': False,
  'lucene_version': '9.3.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [18]:
from tqdm import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:26<00:00, 35.49it/s]


**Answer**: index

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [23]:
def _merge_dicts(a: dict, b: dict, path=[]):
    for key in b:
        if key in a:
            if isinstance(a[key], dict) and isinstance(b[key], dict):
                _merge_dicts(a[key], b[key], path + [str(key)])
            elif a[key] != b[key]:
                raise Exception('Conflict at ' + '.'.join(path + [str(key)]))
        else:
            a[key] = b[key]
    return a
    
def search(q, index = index_name, es_client=es_client, size=5, fields=None, type_match="best_fields", query_patch:dict = None):
    fields = fields or ["question^4", "text"]
    query_patch = query_patch or {}
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": fields,
                        "type": type_match
                    }
                }
            }
        }
    }
    search_query = _merge_dicts(search_query,query_patch) if query_patch else search_query
    return es_client.search(index=index_name, body=search_query)

In [24]:
query = "How do I execute a command in a running docker container?"

response = search(query)

In [25]:
print(json.dumps(response.body,sort_keys=True, indent=4))
response["hits"]["hits"][0]["_score"]

{
    "_shards": {
        "failed": 0,
        "skipped": 0,
        "successful": 1,
        "total": 1
    },
    "hits": {
        "hits": [
            {
                "_id": "rTvZRZABteXuJDEWzTCq",
                "_index": "course-questions",
                "_score": 84.050095,
                "_source": {
                    "course": "machine-learning-zoomcamp",
                    "question": "How do I debug a docker container?",
                    "section": "5. Deploying Machine Learning Models",
                    "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)"
                }
            },
            {
                "_id": "uzvZRZABteXuJDEWly4B",
                "_index": "course-q

84.050095

**Answer: 84.050095**

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [26]:
query = "How do I execute a command in a running docker container?"

filter_query = {
    "query": {
        "bool": {
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = search(query, size=3, query_patch=filter_query)


In [27]:
print(json.dumps(response.body,sort_keys=True, indent=4))
response["hits"]["hits"][-1]["_source"]["question"]

{
    "_shards": {
        "failed": 0,
        "skipped": 0,
        "successful": 1,
        "total": 1
    },
    "hits": {
        "hits": [
            {
                "_id": "rTvZRZABteXuJDEWzTCq",
                "_index": "course-questions",
                "_score": 84.050095,
                "_source": {
                    "course": "machine-learning-zoomcamp",
                    "question": "How do I debug a docker container?",
                    "section": "5. Deploying Machine Learning Models",
                    "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)"
                }
            },
            {
                "_id": "zDvZRZABteXuJDEW0DD2",
                "_index": "course-q

'How do I copy files from a different folder into docker container’s working directory?'

**Answer: 'How do I copy files from a different folder into docker container’s working directory?'**

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [28]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_lines = []

for rsp in response["hits"]["hits"]:
    _source=rsp["_source"]
    question = _source["question"]
    answer = _source['text']

    context_lines.append(context_template.format(question=question, text=answer))


context = '\n\n'.join(context_lines)
context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [29]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context)
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dir

In [30]:
f'Answer: {len(prompt)}'

'Answer: 1462'

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [31]:
!pip install -q tiktoken

In [32]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
f'Answer: {len(encoding.encode(prompt))}'

'Answer: 322'

In [33]:
# encoding.encode(prompt)

In [34]:
# for t in encoding.encode(prompt): 
#     print(t, encoding.decode_single_token_bytes(t))
# encoding.decode(encoding.encode(prompt))

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.



In [35]:
!pip install ollama

In [36]:
!docker run -it -d --rm -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

499e8586eb8d2acec0c53cbeaa17d5363cc97a441f818452b3759758c8bf226e


In [38]:
import ollama

In [41]:
# ollama.chat(model='llama3', messages=[
#   {
#     'role': 'user',
#     'content': "hello",
#   },
# ])

In [40]:
%%time

response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])
print(response['message']['content'])



Based on the context, the answer to your question is:

If the container is already running, execute a command in the specific container:
docker exec -it <container-id> bash
CPU times: user 11.9 ms, sys: 1.24 ms, total: 13.1 ms
Wall time: 1min 56s


In [45]:
response

{'model': 'llama3',
 'created_at': '2024-06-23T16:12:57.918941283Z',
 'message': {'role': 'assistant',
  'content': 'Based on the context, the answer to your question is:\n\nIf the container is already running, execute a command in the specific container:\ndocker exec -it <container-id> bash'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 116271618585,
 'load_duration': 53561412,
 'prompt_eval_count': 329,
 'prompt_eval_duration': 97996022000,
 'eval_count': 37,
 'eval_duration': 18091943000}

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [53]:
input_price = 0.005 / 1000 
output_price = 0.015 / 1000 

def cost(prompt: str, response: str, model:str="gpt-4o",input_price:float=input_price,output_price:float=output_price):
    enc = tiktoken.encoding_for_model(model)
    input_length = len(enc.encode(prompt))
    output_length = len(enc.encode(response))
    
    content_price = input_price * input_length
    response_price = output_price * output_length
    total_price = content_price + response_price

    return total_price

one_request_cost = cost(prompt, response['message']['content'])
print(f"The cost of one request is: {round(one_request_cost, 4)}.")
print(f"The cost of 1000 requests is: {round(one_request_cost*(10**3), 4)}.")


The cost of one request is: 0.0022.
The cost of 1000 requests is: 2.15.
